<p align="center">
  <img src="proj_pics/hh_label.jpg" width="70%">
</p>

<h1><center>Проект: Анализ вакансий из HeadHunter</h1>
   

In [3]:
import pandas as pd
from sqlalchemy import create_engine, text
import requests
from bs4 import BeautifulSoup

In [4]:
# Здесь должна быть информация следующего содержания:
#DBNAME = 
#USER = 
#PASSWORD = 
#HOST = 
#PORT = 

In [5]:
engine = create_engine(f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}')

# <font color='cyan'>1</font>. Предварительный анализ данных

<font color='cyan'>1</font>. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице <font color='orange'>vacancies</font>).

In [6]:
query_1_1 = \
text(f'''

select count(*) Количество_вакансий
from vacancies

''')

with engine.connect() as connection:
    df_1_1 = pd.read_sql_query(query_1_1, connection)
var_1_1 = f'Количество вакансий в нашей базе: {df_1_1.iloc[0, 0]}'
print(var_1_1)

Количество вакансий в нашей базе: 49197


<font color='cyan'>2</font>. Напишите запрос, который посчитает количество работодателей (таблица <font color='orange'>employers</font>).

In [7]:
query_1_2 = \
text(f'''

select count(*) Количество_работодателей
from employers

''')

with engine.connect() as connection:
    df_1_2 = pd.read_sql_query(query_1_2, connection)
var_1_2 = f'Количество работодателей: {df_1_2.iloc[0, 0]}'
print(var_1_2)

Количество работодателей: 23501


<font color='cyan'>3</font>. Посчитате с помощью запроса количество регионов (таблица <font color='orange'>areas</font>).

In [8]:
query_1_3 = \
text(f'''

select count(*) Количество_регионов
from areas

''')

with engine.connect() as connection:
    df_1_3 = pd.read_sql_query(query_1_3, connection)
var_1_3 = f'Количество регионов: {df_1_3.iloc[0, 0]}'
print(var_1_3)

Количество регионов: 1362


<font color='cyan'>4</font>. Посчитате с помощью запроса количество сфер деятельности в базе (таблица <font color='orange'>industries</font>).

In [9]:
query_1_4 = \
text(f'''

select count(*) Количество_сфер
from industries

''')

with engine.connect() as connection:
    df_1_4 = pd.read_sql_query(query_1_4, connection)
var_1_4 = f'Количество сфер дефятельности: {df_1_4.iloc[0, 0]}'
print(var_1_4)

Количество сфер дефятельности: 294


<h2>Выводы:</h2>

- Всего вакансий в нашей базе 49197. Названия вакансий, разумеется, не все уникальные.
- Количество работодателей - 23501 и, так же, встречаются повторяющиеся названия компаний.
- Количество регионов - 1362 и я так понимаю, что там не только регионы указаны, а так же страны, города, посёлки и т.д.
- Количество сфер - 294 и они все уникальные.. Вроде бы.)

---

# <font color='cyan'>2</font>. Детальный анализ вакансий

<font color='cyan'>1</font>. Напишите запрос, который позволит узнать, сколько (<font color='orange'>cnt</font>) вакансий в каждом регионе (<font color='orange'>area</font>).
Отсортируйте по количеству вакансий в порядке убывания.

In [10]:
query_2_1 = \
text(f'''

select
    are.name area,
    count(*) cnt
from
    vacancies vac
    join areas are on are.id = vac.area_id
group by
    are.name
order by 2 desc

''')

with engine.connect() as connection:
    df_2_1 = pd.read_sql_query(query_2_1, connection)
print('Количество вакансий в каждом из регионов:')
df_2_1

Количество вакансий в каждом из регионов:


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


<font color='cyan'>2</font>. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [11]:
query_2_2 = \
text(f'''

select
    count(*)
from
    vacancies
where
    salary_from is not null or salary_to is not null

''')

with engine.connect() as connection:
    df_2_2 = pd.read_sql_query(query_2_2, connection)
var_2_2 = f'У {df_2_2.iloc[0, 0]} вакансий указана хотя бы одна граница зарплатной вилки.'
print(var_2_2)

У 24073 вакансий указана хотя бы одна граница зарплатной вилки.


<font color='cyan'>3</font>. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [12]:
query_2_3 = \
text(f'''

select
    round(avg(salary_from))::int4 avg_from,
    round(avg(salary_to))::int4 avg_to
from
    vacancies
where
    salary_from is not null or salary_to is not null

''')

with engine.connect() as connection:
    df_2_3 = pd.read_sql_query(query_2_3, connection)
var_2_3 = f'Средние границы зарплатной вилки составляют от {df_2_3.iloc[0, 0]} до {df_2_3.iloc[0, 1]} рубликов.'
print(var_2_3)

Средние границы зарплатной вилки составляют от 71065 до 110537 рубликов.


<font color='cyan'>4</font>. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (<font color='orange'>schedule</font>) и типа трудоустройства (<font color='orange'>employment</font>), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [13]:
query_2_4 = \
text(f'''

select
    schedule,
    employment,
    count(*)
from
    vacancies
group by
    schedule, employment
order by
    3 desc

''')

with engine.connect() as connection:
    df_2_4 = pd.read_sql_query(query_2_4, connection)
print('Взаимосвязь типа рабочего графика и типа трудоустройства:')
df_2_4

Взаимосвязь типа рабочего графика и типа трудоустройства:


,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


<font color='cyan'>5</font>. Напишите запрос, выводящий значения поля Требуемый опыт работы (<font color='orange'>experience</font>) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [14]:
query_2_5 = \
text(f'''

select
    experience,
    count(*)
from
    vacancies
group by
    experience
order by
    count(*)

''')

with engine.connect() as connection:
    df_2_5 = pd.read_sql_query(query_2_5, connection)
print('Количество вакансий в зависимости от опыта:')
df_2_5

Количество вакансий в зависимости от опыта:


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


<h2>Выводы:</h2>

Да всё типичное:
- Москва = лучший город земли;
- подавляющее большинство работодателей хочет видеть народ в офисе 9-18 и
- готовы платить в среднем от 70 до 110,
- а добрая половина этой базы вообще не указала «чё по баблу»;
- ну и опыт «от года до 6 лет».

Всё, как обычно: требуются опытные, офисные профессионалы, за маленькую копеечку.

---

# <font color='cyan'>3</font>. Анализ работодателей

<font color='cyan'>1</font>. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

Так, здесь я сначала думал написать через union c офсет-лимитом, но потом передумал и решил оставить себе на память следующий запрос:

In [15]:
query_3_1 = \
text(f'''

select
    emp.name,
    subquery.count
from
    employers emp
    join (
      select
          emp.id,
          count(*),
          row_number() over (order by count(*) desc) row_num --вот это прикольная тема, которая создаёт столбец с подобием индексов;
      from
          employers emp
          join vacancies vac ON emp.id = vac.employer_id
      group by emp.id
    ) subquery on emp.id = subquery.id
where
    subquery.row_num between 1 and 5 --in (1, 5) --тут я вывел топ 5, но по заданию надо было 1 и 5, нужно заменить between на in и будет яндекс с газпромом;
order by 2 desc

''')

with engine.connect() as connection:
    df_3_1 = pd.read_sql_query(query_3_1, connection)
print('Общее количество вакансий:')
display(df_3_1)

Общее количество вакансий:


,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


<font color='cyan'>2</font>. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


Отзыв: жестокое задание. А жестокое оно потому, что.. Когда мы создаём итоговый запрос, мы запускаем промежуточные, а тут оно виснет, если не написать `where vac.id is null`. Хоть опыт в написании запросов у меня и не большой, но это мой первый запрос, после запуска которого можно пойти пить чай, пока он отработает.  
И.. я это задание вообще не понял. Код из пачки от Ильи М.

In [16]:
query_3_2 = \
text(f'''

select
    are.name,
    count(emp.id) emp_cnt,
    count(vac.id) vac_cnt
from
    areas are
    left join employers emp on are.id = emp.area
    left join vacancies vac on vac.area_id = are.id
where
    vac.id is null
group by are.id
order by emp_cnt desc

''')

with engine.connect() as connection:
    df_3_2 = pd.read_sql_query(query_3_2, connection)
df_3_2.head(5)

,name,emp_cnt,vac_cnt
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0


<font color='cyan'>3</font>. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [17]:
query_3_3 = \
text(f'''

select
    emp.name,
    count(distinct vac.area_id)
from
    employers emp
    join vacancies vac on vac.employer_id = emp.id
group by
    emp.id
order by
    2 desc

''')

with engine.connect() as connection:
    df_3_3 = pd.read_sql_query(query_3_3, connection)
print('Количество захваченных регионов:')
df_3_3.head(5)

Количество захваченных регионов:


,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


<font color='cyan'>4</font>. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [18]:
query_3_4 = \
text(f'''

select
    count(*)
from
    employers emp
    left join employers_industries ei on emp.id = ei.employer_id
where
    employer_id is null

''')

with engine.connect() as connection:
    df_3_4 = pd.read_sql_query(query_3_4, connection)
print(f'У {df_3_4.iloc[0, 0]} работодателей не указана сфера деятельности.')

У 8419 работодателей не указана сфера деятельности.


<font color='cyan'>5</font>. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [19]:
query_3_5 = \
text(f'''

select
    name
from
    employers emp
    join employers_industries ei on emp.id = ei.employer_id
group by
    id
having
    count(*) = 4
order by 1
offset 2 limit 1

''')

with engine.connect() as connection:
    df_3_5 = pd.read_sql_query(query_3_5, connection)
print(f'На третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности находится..... {df_3_5.iloc[0, 0]}!!')

На третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности находится..... 2ГИС!!


<font color='cyan'>6</font>. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».


In [20]:
query_3_6 = \
text(f'''

select
    count(*)
from
    employers emp
    join employers_industries ei on emp.id = ei.employer_id
    join industries ind ON ind.id = ei.industry_id
where
    lower(ind.name) like '%разработка программного обеспечения%'

''')

with engine.connect() as connection:
    df_3_6 = pd.read_sql_query(query_3_6, connection)
print(f'{df_3_6.iloc[0, 0]} компаний разрабатывают программное обеспечение.')

3553 компаний разрабатывают программное обеспечение.


<font color='cyan'>7</font>. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8).

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API.

In [21]:
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')

# Да, тут кортеж, хз почему sql не любит списки... Не углублялся.
list_of_mil_cities = tuple([i.text for i in page.find('table', {'class': 'sortable'}).find_all('a')[4:]]) # Мой второй вариант, используя гпт-чат. Оставлю его.

#[i.text for i in page.find_all('a')[52:68]] # Мой первый вариант, руководствуясь материалами модуля PYTHON-17.
#[i.text for i in page.select('div table:nth-of-type(2) tbody tr td a')] # Не мой вариант, но классный, хрен додумаешься до такого.
#[i.text for i in page.select('table.standard.sortable')[0].select('a[href][title]')[2:]] # Ещё 1 не мой вариант.

In [22]:
query_3_7 = \
text(f'''

with subquery_table_3_7 as (
select
    are.name city_name,
    count(*)::int4 cnt
from
    vacancies vac
    join areas are on are.id = vac.area_id
    join employers emp on emp.id = vac.employer_id
where
    emp.name = 'Яндекс' and are.name in {list_of_mil_cities} -- если заменить первое условие на lower(emp.name) like '%яндекс%' - будет немного другой результат.)
group by are.id
order by 2 desc -- Прошу простить, но сортировка desc - более приятная.
)

select 'Total' city_name, sum(cnt)::int4 cnt
from subquery_table_3_7
union all
select *
from subquery_table_3_7

''')

with engine.connect() as connection:
    df_3_7 = pd.read_sql_query(query_3_7, connection)
print('Количество вакансий Яндекса по городам-миллионникам:')
df_3_7

Количество вакансий Яндекса по городам-миллионникам:


,city_name,cnt
0,Total,485
1,Москва,54
2,Санкт-Петербург,42
3,Екатеринбург,39
4,Нижний Новгород,36
5,Новосибирск,35
6,Воронеж,32
7,Краснодар,30
8,Самара,26
9,Уфа,26


<h2>Выводы:</h2>

Я не знаю, что писать. Да, у меня тоже домашняя страница ya.ru.
- У Яндекса в 4 раза больше вакансий, чем у ближайшего сородича по созданию рабочих мест;
- Согласно [количеству регионов](https://ru.wikipedia.org/wiki/%D0%A1%D1%83%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%B9_%D0%A4%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B8) РФ (89), Яндекс(181) захватил ещё несколько соседних стран, как и Ростелеком(152), а так же стоит отметить, какой Денис Иваныч - молодец.)
- 8419 компаний пока ещё не осознают, что работают на Яндекс;
- реклама 2ГИС;
- 3553 компаний скорее всего осознают, что работают на Яндекс или это уже дочки Яндекса;
- Яндекс повсюду.

---

# <font color='cyan'>4</font>. Предметный анализ

<font color='cyan'>1</font>. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.*


In [23]:
query_4_1 = \
text(f'''

select count(*)
from vacancies
where lower(name) like '%data%' or lower(name) like '%данн%'

''')

with engine.connect() as connection:
    df_4_1 = pd.read_sql_query(query_4_1, connection)
print(f'{df_4_1.iloc[0, 0]} вакансий имеет отношение к данным.')

1771 вакансий имеет отношение к данным.


<font color='cyan'>2</font>. Сколько есть подходящих вакансий для начинающего дата-сайентиста?
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.


In [24]:
query_4_2 = \
text(f'''

select count(*)
from vacancies

where (lower(name) like '%data scientist%'
   or lower(name) like '%data science%'
   or (name like '%ML%' and lower(name) not like '%html%')
   or lower(name) like '%исследователь данных%'
   or lower(name) like '%machine learning%'
   or lower(name) like '%машинн%обучен%')

  and (lower(name) like '%junior%'
   or experience = 'Нет опыта'
   or employment = 'Стажировка')

''')

with engine.connect() as connection:
    df_4_2 = pd.read_sql_query(query_4_2, connection)
print(f'{df_4_2.iloc[0, 0]} вакансия для совсем зелёного DS новичка без опыта работы.')

51 вакансия для совсем зелёного DS новичка без опыта работы.


<font color='cyan'>3</font>. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [25]:
query_4_3 = \
text(f'''

select count(*)
from vacancies

where (lower(name) like '%data scientist%'
   or lower(name) like '%data science%'
   or (name like '%ML%' and lower(name) not like '%html%')
   or lower(name) like '%исследователь данных%'
   or lower(name) like '%machine learning%'
   or lower(name) like '%машинн%обучен%')

  and (lower(key_skills) like '%sql%'
   or lower(key_skills) like '%postgress%')

''')

with engine.connect() as connection:
    df_4_3 = pd.read_sql_query(query_4_3, connection)
print(f'{df_4_3.iloc[0, 0]} вакансия для DS специалиста с ключевым навыком SQL/PGSQL.')

201 вакансия для DS специалиста с ключевым навыком SQL/PGSQL.


<font color='cyan'>4</font>. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [26]:
query_4_4 = \
text(f'''

select count(*)
from vacancies

where (lower(name) like '%data scientist%'
   or lower(name) like '%data science%'
   or (name like '%ML%' and lower(name) not like '%html%')
   or lower(name) like '%исследователь данных%'
   or lower(name) like '%machine learning%'
   or lower(name) like '%машинн%обучен%')

  and lower(key_skills) like '%python%'

''')

with engine.connect() as connection:
    df_4_4 = pd.read_sql_query(query_4_4, connection)
print(f'{df_4_4.iloc[0, 0]} вакансия для DS специалиста с ключевым навыком Python.')

351 вакансия для DS специалиста с ключевым навыком Python.


<font color='cyan'>5</font>. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [27]:
query_4_5 = \
text(f'''

select
    round(avg(length(key_skills) - length(replace(key_skills, CHR(9), '')) + 1), 2)

from vacancies

where (lower(name) like '%data scientist%'
   or lower(name) like '%data science%'
   or (name like '%ML%' and lower(name) not like '%html%')
   or lower(name) like '%исследователь данных%'
   or lower(name) like '%machine learning%'
   or lower(name) like '%машинн%обучен%')

''')

with engine.connect() as connection:
    df_4_5 = pd.read_sql_query(query_4_5, connection)
print(f'В среднем в вакансиях указывают {df_4_5.iloc[0, 0]} ключевых навыков.')

В среднем в вакансиях указывают 6.41 ключевых навыков.


<font color='cyan'>6</font>. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*).

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа.

In [28]:
query_4_6 = \
text(f'''

select
    experience,
    round(avg(case
                  when salary_from is not null and salary_to is not null then (salary_from + salary_to) / 2
                  when salary_from is null then salary_to
                  when salary_to is null then salary_from
                  else 0
              end))::int4 avg_salary,
    round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to)))::int4 avg_salary_2 --Оставлю для себя. Я б до такого не додумался.

from vacancies

where (lower(name) like '%data scientist%'
   or lower(name) like '%data science%'
   or (name like '%ML%' and lower(name) not like '%html%')
   or lower(name) like '%исследователь данных%'
   or lower(name) like '%machine learning%'
   or lower(name) like '%машинн%обучен%')

  and (salary_from is not null or salary_to is not null)

group by experience

''')

with engine.connect() as connection:
    df_4_6 = pd.read_sql_query(query_4_6, connection)
print(f'На {df_4_6.iloc[-1, -1]} тыс.руб. в среднем может рассчитывать дата саентист с опытом работы {df_4_6.iloc[-1, 0].lower()}.')
df_4_6

На 243115 тыс.руб. в среднем может рассчитывать дата саентист с опытом работы от 3 до 6 лет.


,experience,avg_salary,avg_salary_2
0,Нет опыта,74643,74643
1,От 1 года до 3 лет,139675,139675
2,От 3 до 6 лет,243115,243115


<h2>Выводы:</h2>

Исходя из того, что мы тут разбирали профессию ДатаСаентист, можно заключить, что голодным такой специалист не останется. 1771 вакансия..  
Постажироваться и набраться опыта можно в 51 месте, по-SQL-ить в 201-м, и по-питонить в 351-м месте.) Хотя, на самом деле там 3.5 тыщи мест, где можно погонять змеюку...))  
Ну короче, я пока не дошёл в курсе до места, где учат грамотно анализировать подобные моменты.

***

# Общий вывод по проекту

Знаете, всегда интересно, кто больше всех востребован и сколько в среднем за это платят, и сколько надо опыта...  
Ну и одни айтишники в топе. А 1С сфера вообще кричит, что ей не хватает специалистов. Их нужно почти столько же, сколько сисадминов. И это исходя из того, что мы учитываем только 24к вакансий из 49к. Считай половину. И первые топ 20 из ИТ сферы, хотя там 294 сферы.  
...  
Какая-то как обычно база, одни айтишники в топе.))  
...  
И, к сожалению, диапазон фантазии у меня не большой и я не знаю, как люди придумывают ещё по 10 дополнительных исследований (я просто нагуглил ещё несколько работ и там была одна, где по 10 штук допов после каждой главы).. Скорее всего это приходит с опытом, которого у меня на данный момент - нет.  
Потому сделал, как сделал.) Старался.) Нет, не навернуть базу.))

In [29]:
query_4_7 = \
text(f'''

select
    name Название_вакансии,
    count(*) Количество_вакансий,
    round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to)))::int4 Средняя_зп,
    mode() within group (order by experience) Мода_требуемого_опыта -- надеюсь я правильно написал моду..
from vacancies
where salary_from is not null or salary_to is not null
group by name
order by 2 desc

''')

with engine.connect() as connection:
    df_4_7 = pd.read_sql_query(query_4_7, connection)

In [31]:
print(var_1_1)
print(var_1_4)
print(var_2_2)
print(var_2_3)
df_4_7.head(20)

Количество вакансий в нашей базе: 49197
Количество сфер дефятельности: 294
У 24073 вакансий указана хотя бы одна граница зарплатной вилки.
Средние границы зарплатной вилки составляют от 71065 до 110537 рубликов.


,Название_вакансии,Количество_вакансий,Средняя_зп,Мода_требуемого_опыта
0,Системный администратор,1287,50115,От 1 года до 3 лет
1,Программист 1С,729,110189,От 1 года до 3 лет
2,Специалист технической поддержки,335,43211,От 1 года до 3 лет
3,Инженер-программист,279,50478,От 1 года до 3 лет
4,Программист 1C,228,96906,От 1 года до 3 лет
5,Аналитик,197,61197,От 1 года до 3 лет
6,Программист,169,60817,От 1 года до 3 лет
7,Специалист службы поддержки в Яндекс Бизнес,168,26000,Нет опыта
8,Бизнес-аналитик,159,92228,От 1 года до 3 лет
9,Инженер технической поддержки,140,48011,От 1 года до 3 лет
